In [3]:
import pandas as pd
import datetime as dt
import numpy as np
import pyproj

SyntaxError: invalid syntax (<ipython-input-3-5ff198d705b4>, line 4)

In [75]:
multiple =1

dummy_df = pd.DataFrame({
    'STATION' : ['dummy 1', 'dummy 2', 'dummy 3', 'dummy 4', 'dummy 5']*multiple,
    'PEOPLE' : [np.random.randint(0,500) for i in range(5*multiple)],
    'DAY': [np.random.randint(0,6) for i in range(5*multiple)],
    'TIME' : [np.random.randint(0,5)*4 for i in range(5*multiple)],
    'LAT': [40.775149, 40.77481, 40.775025, 40.774938, 40.770313]*multiple,
    'LON': [-73.912074, -73.912151, -73.911891, -73.912337, -73.912337]*multiple
})



project_projection = pyproj.Proj("+init=EPSG:4326")  # wgs84
google_projection = pyproj.Proj("+init=EPSG:3857")  # default google projection

lon, lat = pyproj.transform(project_projection, google_projection, longitude, latitude)

dummy_df.head()



,STATION,PEOPLE,DAY,TIME,LAT,LON
0,dummy 1,375,5,4,40.775149,-73.912074
1,dummy 2,473,4,0,40.774810,-73.912151
2,dummy 3,338,4,8,40.775025,-73.911891
3,dummy 4,103,2,16,40.774938,-73.912337
4,dummy 5,499,3,4,40.770313,-73.912337


In [77]:
xs = []
ys = []

for row in dummy_df.itertuples():
    lat =  row[5]
    lon = row[6]
    x, y = pyproj.transform(project_projection, google_projection, lon, lat)
    xs.append(x)
    ys.append(y)

dummy_df['X_CORD']=xs
dummy_df['Y_CORD']=ys

dummy_df.sample(5)


,STATION,PEOPLE,DAY,TIME,LAT,LON,X_CORD,Y_CORD
4,dummy 5,499,3,4,40.770313,-73.912337,-8.227884e+06,4.978522e+06
3,dummy 4,103,2,16,40.774938,-73.912337,-8.227884e+06,4.979202e+06
1,dummy 2,473,4,0,40.774810,-73.912151,-8.227863e+06,4.979183e+06
2,dummy 3,338,4,8,40.775025,-73.911891,-8.227834e+06,4.979214e+06
0,dummy 1,375,5,4,40.775149,-73.912074,-8.227854e+06,4.979233e+06


In [89]:
import bokeh.plotting

from bokeh.plotting import figure, show, output_file
from bokeh.tile_providers import CARTODBPOSITRON
from bokeh.models import ColumnDataSource, ColorBar, HoverTool, LinearColorMapper
from bokeh.palettes import *

bokeh.plotting.reset_output()

output_file("tile.html")

mapper = LinearColorMapper(palette=Plasma256, low=0, high=500)

data = ColumnDataSource(dummy_df)
hover_tool = HoverTool(tooltips =[
    ('Station', '@STATION')
])

ny_longitude = -73.912074
ny_latitude = 40.775149

ny_lon, ny_lat = pyproj.transform(project_projection, google_projection, ny_longitude, ny_latitude)

print(ny_lon, ny_lat)

# range bounds supplied in web mercator coordinates
p = figure(x_range=(ny_lon-10000, ny_lon+10000), y_range=(ny_lat-10000, ny_lat+10000),
           x_axis_type="mercator", y_axis_type="mercator")
p.circle(source=data, x='X_CORD', y='Y_CORD', legend='STATION', 
         color={'field':'PEOPLE', 'transform': mapper}, size=12)
p.add_tile(CARTODBPOSITRON)
p.add_tools(hover_tool)
color_bar = ColorBar(color_mapper=mapper, width=8,  location=(0,0))
p.add_layout(color_bar, 'right')

show(p)

-8227854.441154755 4979232.550263305


(5008370.868820031, -12480129.899747571)